In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import json
import seaborn as sns
import numpy as np
plt.tight_layout()

<Figure size 432x288 with 0 Axes>

In [2]:
logfile_name = "importances_per_omics_all_attributions_extern.json"
result_path = Path('..', 'results')
result_path_moli = result_path / 'moli' 
result_path_super_felt = result_path / 'super_felt' 
result_path_early_integration = result_path / 'early_integration' 
result_path_stacking = result_path / 'stacking' 
result_path_moma = result_path / 'moma' 
result_path_omi_embed = result_path / 'omi_embed' 
result_path_pca = result_path / 'pca' 

result_paths = [result_path_moli, result_path_super_felt, result_path_early_integration,
               result_path_stacking, result_path_moma, result_path_omi_embed, result_path_pca]
method_folder = ['moli_shapley', 'super_felt_shapley', 'stacking_shapley',
       'moma_shapley', 'early_integration_shapley', 'omiEmbed_shapley', 'pca_shapley']
Path("attributions").mkdir(exist_ok=True)
x = ["CNA", "Mutation",  "Expression"]

In [9]:
method_data_sum = {}
method_data_mean = {}

max_sum = 0
max_mean = 0

max_std_sum = 0
max_std_mean = 0

for result_path in result_paths:
    result_path_explanation = result_path / "explanation"
    expression_attributions = []
    mutation_attributions = []
    cna_attributions = []
    
    expression_attributions_mean = []
    mutation_attributions_mean = []
    cna_attributions_mean = []
    
    for logfile_folder in method_folder:
        log_path = result_path_explanation / logfile_folder 
        if log_path.exists():
            drug_paths =  [x for x in log_path.iterdir()]
            for drug_path in drug_paths:
                drug = drug_path.stem
                file_path = log_path / drug / logfile_name
                if file_path.is_file():
                    with open(file_path, 'r') as log_file:
                        data = json.load(log_file)
                        expression_attributions.append(data["sum_expression_importance"])
                        mutation_attributions.append(data["sum_mutation_importance"])
                        cna_attributions.append(data["sum_cna_importance"])
                        
                        expression_attributions_mean.append(data["mean_expression_importance"])
                        mutation_attributions_mean.append(data["mean_mutation_importance"])
                        cna_attributions_mean.append(data["mean_cna_importance"])
                        
    expression_mean_sum = np.nanmean(expression_attributions)
    mutation_mean_sum = np.nanmean(mutation_attributions)
    cna_mean_sum = np.nanmean(cna_attributions)
    expression_std_sum = np.nanstd(expression_attributions)
    mutation_std_sum = np.nanstd(mutation_attributions)
    cna_std_sum = np.nanstd(cna_attributions)
    
    expression_mean_mean = np.nanmean(expression_attributions_mean)
    mutation_mean_mean = np.nanmean(mutation_attributions_mean)
    cna_mean_mean = np.nanmean(cna_attributions_mean)
    expression_std_mean = np.nanstd(expression_attributions_mean)
    mutation_std_mean = np.nanstd(mutation_attributions_mean)
    cna_std_mean = np.nanstd(cna_attributions_mean)
                        
    method_data_sum[result_path.stem] = (expression_mean_sum, mutation_mean_sum, cna_mean_sum,
                                    expression_std_sum, mutation_std_sum, cna_std_sum)
    method_data_mean[result_path.stem] = (expression_mean_mean, mutation_mean_mean, cna_mean_mean,
                                    expression_std_mean, mutation_std_mean, cna_std_mean)
    
    tmp_max_sum = np.max([expression_mean_sum, mutation_mean_sum, cna_mean_sum,])
    if tmp_max_sum > max_sum:
        max_sum = tmp_max_sum
        
    tmp_max_mean = np.max([expression_mean_mean, mutation_mean_mean, cna_mean_mean,])
    if tmp_max_mean > max_mean:
        max_mean = tmp_max_mean
        
    tmp_max_sum_sd = np.max([expression_std_sum, mutation_std_sum, cna_std_sum,])
    if tmp_max_sum_sd > max_std_sum:
        max_std_sum = tmp_max_sum_sd
        
    tmp_max_mean_sd = np.max([expression_std_mean, mutation_std_mean, cna_std_mean,])
    if tmp_max_mean_sd > max_std_mean:
        max_std_mean = tmp_max_mean_sd

In [10]:
beautify_dict = {'moli': "MOLI", 'super_felt': "Super.FELT" , 'early_integration': "Early Integration",
                 'stacking': "Omics Stacking" , 'moma':"MOMA" ,'omi_embed':"OmiEmbed",
                 'pca':"PCA"}

In [13]:
for method, data in method_data_sum.items():
    expression_mean_sum = data[0]
    mutation_mean_sum = data[1]
    cna_mean_sum = data[2]
    expression_std_sum = data[3]
    mutation_std_sum = data[4]
    cna_std_sum = data[5]
    cna_lower_std = cna_std_sum if (cna_mean_sum - cna_std_sum) > 0 else cna_mean_sum
    mutation_lower_std = mutation_std_sum if (mutation_mean_sum - mutation_std_sum) > 0 else mutation_mean_sum
    expression_lower_std = expression_std_sum if (expression_mean_sum - expression_std_sum) > 0 else expression_mean_sum
    plt.barh(x, [cna_mean_sum, mutation_mean_sum, expression_mean_sum], 
             xerr=[[cna_lower_std, mutation_lower_std, expression_lower_std], 
                   [cna_std_sum, mutation_std_sum, expression_std_sum]])
    plt.xlabel("Summarized Attribution")
    plt.title(beautify_dict[method])
    plt.xlim([0, max_sum+max_std_sum])
    plt.savefig(f"attributions/{method}_sum.pdf", bbox_inches='tight')
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [14]:
for method, data in method_data_mean.items():
    expression_mean_mean = data[0]
    mutation_mean_mean = data[1]
    cna_mean_mean = data[2]
    expression_std_mean = data[3]
    mutation_std_mean = data[4]
    cna_std_mean = data[5]
    
    cna_lower_std = cna_std_mean if (cna_mean_mean - cna_std_mean) > 0 else cna_mean_mean
    mutation_lower_std = mutation_std_mean if (mutation_mean_mean - mutation_std_mean) > 0 else mutation_mean_mean
    expression_lower_std = expression_std_mean if (expression_mean_mean - expression_std_mean) > 0 else expression_mean_mean
    
    plt.barh(x, [cna_mean_mean, mutation_mean_mean, expression_mean_mean], 
             xerr=[[cna_lower_std, mutation_lower_std, expression_lower_std], 
                   [cna_std_mean, mutation_std_mean, expression_std_mean]])
    plt.title(beautify_dict[method])
    plt.xlim([0, max_mean+max_std_mean])
    plt.xlabel("Mean Attribution")
    plt.savefig(f"attributions/{method}_mean.pdf", bbox_inches='tight')
    plt.clf()

<Figure size 432x288 with 0 Axes>